In [1]:
import json
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from src.utils.UsefulPaths import Paths
from sklearn.cluster import KMeans
from src.utils.KnnUtils import OptimalCluster
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

2023-11-03 17:49:20,241 - DEBUG - matplotlib data path: C:\Users\Thiago\PycharmProjects\genome\venv\Lib\site-packages\matplotlib\mpl-data
2023-11-03 17:49:20,246 - DEBUG - CONFIGDIR=C:\Users\Thiago\.matplotlib
2023-11-03 17:49:20,247 - DEBUG - interactive is False
2023-11-03 17:49:20,248 - DEBUG - platform is win32
2023-11-03 17:49:20,277 - DEBUG - CACHEDIR=C:\Users\Thiago\.matplotlib
2023-11-03 17:49:20,279 - DEBUG - Using fontManager instance from C:\Users\Thiago\.matplotlib\fontlist-v330.json


In [2]:
paths = Paths()

with open(paths.json_subsectors, 'r') as file:
    subsectors = json.load(file)

In [3]:
subsectors_class = list(subsectors.keys())
subsectors_class.append('Others')

for subsector in subsectors_class:
    print(f'{subsector}')
    
print(f'\nTotal: {len(subsectors_class)}')

Artificial Intelligence; Big Data and Analytics
Advanced Manufacturing and Robotics
Clean Technology
Financial Technology
Blockchain
Cybersecurity
Agriculture Technology
New Food
Advertising Technology
Blue Economy
Digital Media
Gaming
Augmented Reality; Virtual Reality
Educational Technology
Industry 4.0
Biopharmaceutical; Biotechonology
Medical Technology; Medical devices
Others

Total: 18


In [4]:
df_abstract_patents = pd.read_csv(os.path.join(paths.data_processed, 'abstract_patents.csv'))

In [5]:
slice_number = 1000
df_sliced = df_abstract_patents[:slice_number]
df_sliced.to_csv(os.path.join(paths.data_processed, 'abstract_patents_sliced.csv'), index=False)
df_sliced

,publication_number,abstract,word_count,patent_id,app_name,app_year,city,country,country_name,ecosystem,grant_date,grant_year,app_date,class_concat,class_distinct_count,class_ipc_concat,class_ipc_distinct_count
0,20180152842,"A method, and a mobile device adapted thereto,...",80,20180152842,Moorim Kim,2018,Seoul,KR,"Korea, Republic of",Seoul,2018-05-31 00:00:00+00:00,2018,2018-01-29 00:00:00+00:00,"H04M,H04W,G06F",3.00,"G06F,H04M,H04W",3.00
1,20180140889,A dumbbell with a selectable number of weight ...,131,20180140889,Per HÖGLUND,2018,Jonkoping,SE,Sweden,Jonkoping,2018-05-24 00:00:00+00:00,2018,2018-01-03 00:00:00+00:00,A63B,1.00,A63B,1.00
2,20180143803,A virtual assistant AI system that may be conn...,83,20180143803,David Placa,2018,San Francisco,US,United States,Silicon Valley,2018-05-24 00:00:00+00:00,2018,2018-01-16 00:00:00+00:00,G06F,1.00,G06F,1.00
3,20180169402,Connector assemblies that are separate from me...,101,20180169402,Patrick Wells,2018,Columbia Heights,US,United States,Minneapolis,2018-06-21 00:00:00+00:00,2018,2018-02-15 00:00:00+00:00,"H01R,A61N",2.00,"A61N,H01R",2.00
4,20180163931,"An LED tube lamp, comprising a lamp tube, whic...",130,20180163931,Aiming Xiong,2018,Jiaxing,CN,China,Hangzhou,2018-06-14 00:00:00+00:00,2018,2018-02-06 00:00:00+00:00,"F21Y,F21K,F21V,H05B",4.00,"F21K,F21V,F21Y,H05B",4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,20180128819,Methods for preparation of cells for analysis ...,112,20180128819,Jita De,2018,Costa Mesa,US,United States,Los Angeles,2018-05-10 00:00:00+00:00,2018,2018-01-08 00:00:00+00:00,G01N,1.00,G01N,1.00
996,20180171674,The present disclosure provides lockable cover...,80,20180171674,Shadi AbuGhazaleh,2018,Guilford,US,United States,Hartford,2018-06-21 00:00:00+00:00,2018,2018-02-15 00:00:00+00:00,"E05B,H05K",2.00,"E05B,H05K",2.00
997,20180159626,A data communication system comprises an optic...,145,20180159626,Donald Chang,2018,Thousand Oaks,US,United States,Los Angeles,2018-06-07 00:00:00+00:00,2018,2018-01-29 00:00:00+00:00,"H04J,H04B,H04Q",3.00,"H04B,H04J,H04Q",3.00
998,20180130519,Systems are provided for a three dimension sta...,165,20180130519,Chien-Yuan Chen,2018,Hsinchu,TW,"Taiwan, Province of China",Taipei City,2018-05-10 00:00:00+00:00,2018,2018-01-09 00:00:00+00:00,"H01L,G11C",2.00,"G11C,H01L",2.00


In [6]:
embedder = SentenceTransformer('distilbert-base-uncased')

2023-11-03 17:49:23,625 - INFO - Load pretrained SentenceTransformer: distilbert-base-uncased
2023-11-03 17:49:23,627 - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2023-11-03 17:49:24,088 - DEBUG - https://huggingface.co:443 "GET /api/models/distilbert-base-uncased HTTP/1.1" 200 8027
2023-11-03 17:49:24,257 - DEBUG - https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/.gitattributes HTTP/1.1" 200 0
2023-11-03 17:49:24,419 - DEBUG - https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/LICENSE HTTP/1.1" 200 0
2023-11-03 17:49:24,580 - DEBUG - https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/README.md HTTP/1.1" 200 0
2023-11-03 17:49:24,746 - DEBUG - https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/config.json HTTP/1.1" 200 0
2023-11-03 17:49:24,911 

In [7]:
sentences = list(df_sliced.abstract)

In [12]:
query = f'I have a patent abstract "{sentences[0]}". Among the following subcategories: {list(subsectors.keys())}, which would be the most appropriate to classify this patent?'

C:\Users\Thiago\AppData\Local\Temp\ipykernel_6016\103647595.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sliced['query'] = df_sliced['abstract'].apply(create_query_sentence)


,publication_number,abstract,word_count,patent_id,app_name,app_year,city,country,country_name,ecosystem,grant_date,grant_year,app_date,class_concat,class_distinct_count,class_ipc_concat,class_ipc_distinct_count,query
0,20180152842,"A method, and a mobile device adapted thereto,...",80,20180152842,Moorim Kim,2018,Seoul,KR,"Korea, Republic of",Seoul,2018-05-31 00:00:00+00:00,2018,2018-01-29 00:00:00+00:00,"H04M,H04W,G06F",3.00,"G06F,H04M,H04W",3.00,"I have a patent abstract 'A method, and a mobi..."
1,20180140889,A dumbbell with a selectable number of weight ...,131,20180140889,Per HÖGLUND,2018,Jonkoping,SE,Sweden,Jonkoping,2018-05-24 00:00:00+00:00,2018,2018-01-03 00:00:00+00:00,A63B,1.00,A63B,1.00,I have a patent abstract 'A dumbbell with a se...
2,20180143803,A virtual assistant AI system that may be conn...,83,20180143803,David Placa,2018,San Francisco,US,United States,Silicon Valley,2018-05-24 00:00:00+00:00,2018,2018-01-16 00:00:00+00:00,G06F,1.00,G06F,1.00,I have a patent abstract 'A virtual assistant ...
3,20180169402,Connector assemblies that are separate from me...,101,20180169402,Patrick Wells,2018,Columbia Heights,US,United States,Minneapolis,2018-06-21 00:00:00+00:00,2018,2018-02-15 00:00:00+00:00,"H01R,A61N",2.00,"A61N,H01R",2.00,I have a patent abstract 'Connector assemblies...
4,20180163931,"An LED tube lamp, comprising a lamp tube, whic...",130,20180163931,Aiming Xiong,2018,Jiaxing,CN,China,Hangzhou,2018-06-14 00:00:00+00:00,2018,2018-02-06 00:00:00+00:00,"F21Y,F21K,F21V,H05B",4.00,"F21K,F21V,F21Y,H05B",4.00,"I have a patent abstract 'An LED tube lamp, co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,20180128819,Methods for preparation of cells for analysis ...,112,20180128819,Jita De,2018,Costa Mesa,US,United States,Los Angeles,2018-05-10 00:00:00+00:00,2018,2018-01-08 00:00:00+00:00,G01N,1.00,G01N,1.00,I have a patent abstract 'Methods for preparat...
996,20180171674,The present disclosure provides lockable cover...,80,20180171674,Shadi AbuGhazaleh,2018,Guilford,US,United States,Hartford,2018-06-21 00:00:00+00:00,2018,2018-02-15 00:00:00+00:00,"E05B,H05K",2.00,"E05B,H05K",2.00,I have a patent abstract 'The present disclosu...
997,20180159626,A data communication system comprises an optic...,145,20180159626,Donald Chang,2018,Thousand Oaks,US,United States,Los Angeles,2018-06-07 00:00:00+00:00,2018,2018-01-29 00:00:00+00:00,"H04J,H04B,H04Q",3.00,"H04B,H04J,H04Q",3.00,I have a patent abstract 'A data communication...
998,20180130519,Systems are provided for a three dimension sta...,165,20180130519,Chien-Yuan Chen,2018,Hsinchu,TW,"Taiwan, Province of China",Taipei City,2018-05-10 00:00:00+00:00,2018,2018-01-09 00:00:00+00:00,"H01L,G11C",2.00,"G11C,H01L",2.00,I have a patent abstract 'Systems are provided...


In [16]:
number_of_tokens = len(embedder.encode(query))

print(number_of_tokens)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [19]:
num_tokens = len(embedder.tokenizer.tokenize(query))
max_tokens = embedder.get_max_seq_length()

print(f"Number of tokens in query: {num_tokens}")
print(f"Maximum tokens allowed: {max_tokens}")

Maximum tokens allowed: 512


Batches:   0%|          | 0/1 [00:00<?, ?it/s]